In [22]:
#Load libraries
import pandas as pd
import geopy as geo
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.model_selection import KFold, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
#from tqdm import tqdm  #NOT NEEDED FOR GRADING. just a progress bar for a long step

#Load Data
df = pd.read_csv('default_features_1059_tracks.txt', delimiter=',')

In [24]:
df

,1,2,3,4,5,6,7,8,9,10,...,61,62,63,64,65,66,67,68,latitude,longitude
0,7.161286,7.835325,2.911583,0.984049,-1.499546,-2.094097,0.576000,-1.205671,1.849122,-0.425598,...,-1.504263,0.351267,-1.018726,-0.174878,-1.089543,-0.668840,-0.914772,-0.836250,-15.75,-47.95
1,0.225763,-0.094169,-0.603646,0.497745,0.874036,0.290280,-0.077659,-0.887385,0.432062,-0.093963,...,-0.495712,-0.465077,-0.157861,-0.157189,0.380951,1.088478,-0.123595,1.391141,14.91,-23.51
2,-0.692525,-0.517801,-0.788035,1.214351,-0.907214,0.880213,0.406899,-0.694895,-0.901869,-1.701574,...,-0.637167,0.147260,0.217914,2.718442,0.972919,2.081069,1.375763,1.063847,12.65,-8.00
3,-0.735562,-0.684055,2.058215,0.716328,-0.011393,0.805396,1.497982,0.114752,0.692847,0.052377,...,-0.178325,-0.065059,-0.724247,-1.020687,-0.751380,-0.385005,-0.012326,-0.392197,9.03,38.74
4,0.570272,0.273157,-0.279214,0.083456,1.049331,-0.869295,-0.265858,-0.401676,-0.872639,1.147483,...,-0.919463,-0.667912,-0.820172,-0.190488,0.306974,0.119658,0.271838,1.289783,34.03,-6.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054,0.399577,0.310805,-0.039326,-0.111546,0.304586,-0.943453,0.114960,-0.335898,0.826753,-0.393786,...,-0.558717,0.998897,-0.106835,1.526307,0.646088,2.467278,1.867699,1.719302,-6.17,35.74
1055,1.640386,1.306224,0.192745,-1.816855,-1.311906,-2.128963,-1.875967,0.094232,-1.429742,0.873777,...,0.223143,-0.032425,0.226782,0.182107,0.517466,1.126762,2.220671,4.422651,11.55,104.91
1056,-0.772360,-0.670596,-0.840420,-0.832105,0.277346,1.152162,0.241470,0.229092,0.019036,-0.068804,...,0.449239,-0.965270,-0.590039,-0.804297,0.044170,-0.718175,-0.983640,-0.573822,41.33,19.80
1057,-0.996965,-1.099395,3.515274,-0.508185,-1.102654,0.192081,0.069821,0.264674,-0.411533,0.501164,...,1.941398,1.769292,0.738616,1.240377,-0.546002,-0.137473,-0.781036,-0.832167,54.68,25.31


In [13]:
#Create predictive features and target feature
X = df.drop(['longitude', 'latitude'], axis=1)
y = df[['longitude', 'latitude']]

In [5]:
#Using Random Forest to get feature importance

#Specify Regressor
model = RandomForestRegressor(random_state=77)

#K Folds
kf = KFold(n_splits=5, shuffle=True, random_state=77)
feature_importances = []

for train_index, test_index in kf.split(df):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    model.fit(X_train, y_train)
    
    feature_importances.append(model.feature_importances_)

#AVG importance scores from each fold
mean_importances = np.mean(feature_importances, axis=0)

#Data frame for comparison
importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': mean_importances
}).sort_values(by='Importance', ascending=False)

print(importance_df.head(30))
#the far left is just the index number

   Feature  Importance
4        5    0.076876
19      20    0.051632
54      55    0.032317
3        4    0.029294
38      39    0.029271
20      21    0.025269
8        9    0.024500
7        8    0.024146
21      22    0.023693
55      56    0.022190
22      23    0.021924
37      38    0.019833
53      54    0.019720
57      58    0.018916
51      52    0.018230
35      36    0.016656
52      53    0.016597
24      25    0.016279
43      44    0.015433
39      40    0.015256
36      37    0.015011
1        2    0.014641
26      27    0.014367
56      57    0.014209
17      18    0.014092
6        7    0.014025
25      26    0.013350
44      45    0.012930
59      60    0.012454
5        6    0.012107


In [49]:
#PCA for features with importance >0.015
pca = PCA(n_components = 20)
pca.fit(X)

#Make principal components
X_pca = pca.transform(X)


In [ ]:
#This cell is from ChatGPT cause I couldn't figure out how to actually use the PCA
#Honesty is the best policy 

# Set a threshold for high loadings
loading_threshold = 0.3  # Adjust this based on your analysis

# Get the absolute loadings for selected components
important_loadings = loadings[selected_pcs].abs()

# Identify columns that exceed the loading threshold in any selected component
predictive_columns = important_loadings[(important_loadings > loading_threshold).any(axis=1)].index

# Get the most predictive columns
X_predictive = X[predictive_columns]

In [ ]:
#Random Forest Regression

#Train/Test Split (used for all regressors)
X_train, X_test, y_train, y_test = train_test_split(X_predictive, y, test_size=0.2, random_state=42)

#Specify regressor (77 for my soccer # :) )
model_rf = MultiOutputRegressor(RandomForestRegressor(random_state=77))

#Actually train
model_rf.fit(X_train, y_train)

#Predict
y_pred = model_rf.predict(X_test)

#Evaluate
mse = mean_squared_error(y_test, y_pred, multioutput='raw_values')
print(f"Mean Squared Error for Latitude: {mse[0]}")
print(f"Mean Squared Error for Longitude: {mse[1]}")

r2 = r2_score(y_test, y_pred, multioutput='raw_values')
print(f"R^2 for Latitude: {r2[0]}")
print(f"R^2 for Longitude: {r2[1]}")

In [ ]:
#Specify and train
model_lr = MultiOutputRegressor(LinearRegression())
model_lr.fit(X_train, y_train)

#Predict
y_pred = model_lr.predict(X_test)

#Evaluate
mse = mean_squared_error(y_test, y_pred, multioutput='raw_values')
print(f"Mean Squared Error for Latitude: {mse[0]}")
print(f"Mean Squared Error for Longitude: {mse[1]}")

r2 = r2_score(y_test, y_pred, multioutput='raw_values')
print(f"R^2 for Latitude: {r2[0]}")
print(f"R^2 for Longitude: {r2[1]}")

In [ ]:
#Specify and train
model_kn = MultiOutputRegressor(KNeighborsRegressor(n_neighbors=5))
model_kn.fit(X_train, y_train)

#Predict
y_pred = model_kn.predict(X_test)

#Evaluate
mse = mean_squared_error(y_test, y_pred, multioutput='raw_values')
print(f"Root Mean Squared Error for Latitude: {np.sqrt(mse[0])}")
print(f"Mean Squared Error for Longitude: {mse[1]}")

r2 = r2_score(y_test, y_pred, multioutput='raw_values')
print(f"R^2 for Latitude: {r2[0]}")
print(f"R^2 for Longitude: {r2[1]}")

In [ ]:
#Specify and train
model_gb = MultiOutputRegressor(GradientBoostingRegressor(random_state=77))
model_gb.fit(X_train, y_train)

#Predict
y_pred = model_gb.predict(X_test)

#Evaluate
mse = mean_squared_error(y_test, y_pred, multioutput='raw_values')
print(f"Mean Squared Error for Latitude: {mse[0]}")
print(f"Mean Squared Error for Longitude: {mse[1]}")

r2 = r2_score(y_test, y_pred, multioutput='raw_values')
print(f"R^2 for Latitude: {r2[0]}")
print(f"R^2 for Longitude: {r2[1]}")

In [ ]:
#Specify and train
model_mlp = MultiOutputRegressor(MLPRegressor(hidden_layer_sizes=(200,200,100), max_iter=1000, random_state=77))
model_mlp.fit(X_train, y_train)

#Predict
y_pred = model_mlp.predict(X_test)

#Evaluate
mse = mean_squared_error(y_test, y_pred, multioutput='raw_values')
print(f"Mean Squared Error for Latitude: {mse[0]}")
print(f"Mean Squared Error for Longitude: {mse[1]}")

r2 = r2_score(y_test, y_pred, multioutput='raw_values')
print(f"R^2 for Latitude: {r2[0]}")
print(f"R^2 for Longitude: {r2[1]}")


In [14]:
#Regression 2 electric bugaloo, now featuring distances, K-folds, and Grid Search
#Random Forest
#I commented out the grid search stuff after I got my hyperparamters

# Hyperparameter grid 
#param_grid = {
    #'n_estimators': [25, 50, 100, 150],
    #'max_depth': [5, 10, 15, 20],
    #'min_samples_split': [2, 5, 10],
    #'min_samples_leaf': [2, 4, 6]
#}
#distances_all_splits = []

#Train/test
kf = KFold(n_splits=5, shuffle=True, random_state=77)

for train_index, test_index in kf.split(df):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

 # Initialize regressor with hyperparams
regressor = RandomForestRegressor(max_depth = 20, min_samples_leaf = 2, min_samples_split = 2, n_estimators = 100, random_state=77)
regressor.fit(X_train, y_train)
    #base_regressor = RandomForestRegressor(random_state=77)

    # Perform grid search
    #grid_search = GridSearchCV(
        #estimator=base_regressor,
        #param_grid=param_grid,
        #scoring='neg_mean_squared_error',
        #cv=3,
        #n_jobs=-1
    #)
    #grid_search.fit(X_train, y_train)

    # Use the best estimator
    #best_regressor = grid_search.best_estimator_

#Predict
y_pred = regressor.predict(X_test)

#Calculate distances
distances = []
for actual, predicted in zip(y_test.values, y_pred):
    actual_coords = tuple(actual) 
    predicted_coords = tuple(predicted) 
    distance = geodesic(actual_coords, predicted_coords).kilometers
    distances.append(distance)
    #distances_all_splits.append(distances)

#print(f"Best hyperparameters: {grid_search.best_params_}")
# Output: Best hyperparameters: {'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100}
#DataFrame for viewing pleasure
results = pd.DataFrame({
    'Actual Latitude': y_test['latitude'].values,
    'Actual Longitude': y_test['longitude'].values,
    'Predicted Latitude': y_pred[:, 0],
    'Predicted Longitude': y_pred[:, 1],
    'Distance (km)': distances
})

#Evaluate
print(f"Mean Distance Error: {np.mean(distances):.2f} km")
print(results)

Mean Distance Error: 3781.41 km
     Actual Latitude  Actual Longitude  Predicted Latitude  \
0              38.74              9.03           14.975827   
1              69.21             41.26           18.589968   
2              26.10             44.41           31.607073   
3              26.10             44.41           28.838263   
4              26.10             44.41           26.335484   
..               ...               ...                 ...   
206           139.71             35.70           25.145962   
207            -0.12             52.50           10.469518   
208            44.78             41.71           38.630822   
209            35.74             -6.17           20.237910   
210            19.80             41.33           32.479003   

     Predicted Longitude  Distance (km)  
0              -4.669355    4765.351291  
1              21.704878    5146.337336  
2              35.595534    1645.023318  
3              40.481171    2147.211749  
4            

In [19]:
#Gradient Boosting

#Train/test split
kf = KFold(n_splits=5, shuffle=True, random_state=77)

for train_index, test_index in kf.split(df):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

#Specify Regressor
base_regressor = GradientBoostingRegressor(n_estimators=150, learning_rate=0.1, random_state=77)
multi_output_regressor = MultiOutputRegressor(base_regressor)

#Predict
multi_output_regressor.fit(X_train, y_train)
y_pred = multi_output_regressor.predict(X_test)

#Predict, but also keep predictions in valid lat/lon ranges 
y_pred[:, 0] = np.clip(y_pred[:, 0], -90, 90)  # Latitude
y_pred[:, 1] = np.clip(y_pred[:, 1], -180, 180)  # Longitude

#Calculate distances
gbr_distances = [
    geodesic(actual, predicted).kilometers
    for actual, predicted in zip(y_test.values, gbr_predictions)
]

#Data Frame for viewing
results = pd.DataFrame({
    'Actual Latitude': y_test['longitude'].values,
    'Actual Longitude': y_test['latitude'].values,
    'GBR Latitude': gbr_predictions[:, 0],
    'GBR Longitude': gbr_predictions[:, 1],
    'GBR Distance (km)': gbr_distances,
})

#Evaluate
print(f"Mean Distance Error: {np.mean(gbr_distances):.2f} km")
print(results)

Mean Distance Error: 3773.85 km
     Actual Latitude  Actual Longitude  GBR Latitude  GBR Longitude  \
0               9.03             38.74     18.018115     -16.546236   
1              41.26             69.21     12.289033       8.519622   
2              44.41             26.10     39.750153      32.068618   
3              44.41             26.10     31.358596      30.485784   
4              44.41             26.10     39.041594      50.239472   
..               ...               ...           ...            ...   
206            35.70            139.71     27.336979      39.182597   
207            52.50             -0.12     22.382138      14.109491   
208            41.71             44.78     41.002584      59.463274   
209            -6.17             35.74     15.501280      43.801969   
210            41.33             19.80     26.650248      35.972790   

     GBR Distance (km)  
0          6045.960482  
1          6676.172472  
2           715.024318  
3          1498

In [54]:
#Reverse Geo-locate to get Country and add to df

#import saved df because this took forever to run. Can be downloaded via GitHub
newdf = pd.read_csv('MusicDataWCountry.csv', delimiter=',') 

################################################################################
geolocator = Nominatim(user_agent="myapp")

#Pull address from coords and country name from address
def get_country(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), language='en')
        if location and 'country' in location.raw['address']:
            return location.raw['address']['country']
        return None
    except Exception as e:
        print(f"Error for coordinates ({lat}, {lon}): {e}")
        return None

#Add to df with progress bar for sanity
tqdm.pandas() 
newdf['country'] = df.progress_apply(lambda row: get_country(row['longitude'], row['latitude']), axis=1)

print(newdf)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1059/1059 [09:36<00:00,  1.84it/s]

             1         2         3         4         5         6         7  \
0     7.161286  7.835325  2.911583  0.984049 -1.499546 -2.094097  0.576000   
1     0.225763 -0.094169 -0.603646  0.497745  0.874036  0.290280 -0.077659   
2    -0.692525 -0.517801 -0.788035  1.214351 -0.907214  0.880213  0.406899   
3    -0.735562 -0.684055  2.058215  0.716328 -0.011393  0.805396  1.497982   
4     0.570272  0.273157 -0.279214  0.083456  1.049331 -0.869295 -0.265858   
...        ...       ...       ...       ...       ...       ...       ...   
1054  0.399577  0.310805 -0.039326 -0.111546  0.304586 -0.943453  0.114960   
1055  1.640386  1.306224  0.192745 -1.816855 -1.311906 -2.128963 -1.875967   
1056 -0.772360 -0.670596 -0.840420 -0.832105  0.277346  1.152162  0.241470   
1057 -0.996965 -1.099395  3.515274 -0.508185 -1.102654  0.192081  0.069821   
1058 -0.150911 -0.094333 -0.568885 -0.614652  0.332477 -0.954948 -1.527722   

             8         9        10  ...        63        64    

In [33]:
#saving so I only have to do that ^ once
#newdf.to_csv('MusicDataWCountry.csv')

#K Means Clustering
num_clusters = 7

#Specify and predict
kmeans = KMeans(n_clusters=num_clusters, random_state=77)
cluster_labels = kmeans.fit_predict(X)

#Add to df
newdf['Cluster'] = cluster_labels

#Remove any duplicates
unique_df = newdf[['country', 'Cluster']].drop_duplicates()

#Make table to compare clusters
cluster_table = unique_df.groupby('Cluster')['country'].apply(list).reset_index()
cluster_table = cluster_table.set_index('Cluster').T
print(cluster_table)

Cluster                                                  0  \
country  [Turkey, Egypt, Algeria, Senegal, Kyrgyzstan, ...   

Cluster                                                  1  \
country  [Lithuania, Algeria, Ethiopia, Thailand, China...   

Cluster                                                  2  \
country  [Brazil, Cape Verde, Mali, Ethiopia, Morocco, ...   

Cluster                                                  3  \
country  [Uzbekistan, Algeria, Romania, United Kingdom,...   

Cluster                                                  4  \
country  [Mali, Kyrgyzstan, Kenya, Iran, India, Italy, ...   

Cluster                                                  5  \
country  [Mali, Uzbekistan, Kenya, Greece, Brazil, Cape...   

Cluster                                                  6  
country  [Albania, Egypt, Myanmar, Lithuania, Brazil, I...  


In [31]:
#Decision Tree Classification

#Split new df for new target and get rid of coords bc overfitting
X = newdf.drop(['longitude', 'latitude','country'], axis=1)
y = newdf['country']

#K Folds
kf = KFold(n_splits=5, shuffle=True, random_state=77)
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]    

#Specify classifier
classifier = RandomForestClassifier(random_state=77, n_estimators = 100)

classifier.fit(X_train, y_train)

#Predict
y_pred = classifier.predict(X_test)

#Evaluate
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.44
Confusion Matrix:
[[3 0 0 ... 0 0 0]
 [0 2 0 ... 2 0 0]
 [0 0 1 ... 0 0 1]
 ...
 [0 0 0 ... 5 0 1]
 [0 0 0 ... 0 7 0]
 [0 1 0 ... 0 0 2]]

Classification Report:
                precision    recall  f1-score   support

       Albania       0.50      0.60      0.55         5
       Algeria       0.17      0.25      0.20         8
     Australia       0.25      0.50      0.33         2
        Belize       0.50      0.50      0.50         2
        Brazil       0.17      0.25      0.20         4
      Cambodia       1.00      1.00      1.00         4
    Cape Verde       0.00      0.00      0.00         2
         China       0.40      0.22      0.29         9
         Egypt       0.00      0.00      0.00         9
      Ethiopia       0.60      0.25      0.35        12
       Georgia       0.60      1.00      0.75         6
        Greece       0.75      0.50      0.60         6
         India       0.15      0.30      0.20        10
     Indonesia       0.00      0.00   

C:\Users\shsma\OneDrive\Documents\ML Class\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\shsma\OneDrive\Documents\ML Class\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\shsma\OneDrive\Documents\ML Class\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap